# Prevalence of PRIMIS codelists

In [1]:
import pandas as pd
import numpy as np
import os

suffix = f"_{os.environ.get('OPENSAFELY_BACKEND', 'tpp')}"
os.makedirs(os.path.join("..","safe-outputs"), exist_ok=True)

### Load data

In [2]:
df = pd.read_csv(os.path.join("..","output","input.csv"))

### Create ageband

In [3]:
agebands = ['16-39', '40-69', '70+']
conditions = [
    (df['age'] >= 16) & (df['age'] < 40),
    (df['age'] >= 40) & (df['age'] < 70),
    (df['age'] >= 70) & (df['age'] < 120)]
choices = agebands
df['ageband'] = np.select(conditions, choices, default=np.nan)

# filter to largest sex groups
df['sex'] = df['sex'].replace(['I','U'], np.nan)

### Summarise data

In [4]:
# list columns of interest 
cols_allyears = [c for c in df.columns if c not in ["age","patient_id"]]
cols_recent = ["preg", "pregdel"]

In [5]:
# filter to valid sexes and agegroups only
df1 = df.copy().loc[(df["sex"].isin(["F","M"])) & (df["ageband"].isin(agebands))]

# for codes that are only relevant if recent (pregnancy/delivery), remove any older dates
for c in cols_recent:
    df1[(df1[c]<2020)] = np.nan

# summarise by age and gender and suppress low numbers
out = df1[cols_allyears].groupby(["ageband", "sex"]).count().transpose().replace([0,1,2,3,4],0)
out["total"] = out.sum(axis=1)

out.to_csv(os.path.join("..","safe-outputs",f"code-count-by-age-and-sex{suffix}.csv"))
out

ageband        16-39    40-69    70+    total
sex                F  M     F  M   F  M      
ast                0  0     0  0   0  0     0
astadm             5  0     0  5   0  0    10
astrx              0  0     0  0   0  0     0
bmi                0  0     0  0   0  0     0
bmi_stage          0  0     0  0   0  0     0
carehome           0  0     0  0   0  0     0
carer              0  0     0  0   0  0     0
chd_cov            0  0     0  5   0  0     5
ckd15              0  0     0  0   0  0     0
ckd35              0  0     0  0   0  0     0
ckd_cov            0  0     0  0   0  0     0
cld                0  0     0  0   0  0     0
cns_cov            0  0     0  0   0  0     0
cov1decl           0  0     0  0   0  0     0
cov2decl           5  5     0  0   0  0    10
covadm1            0  0     0  0   0  0     0
covadm2            0  0     0  0   0  0     0
covcontra          0  0     0  0   0  0     0
covrx              0  0     0  0   0  0     0
diab               0  0     0  6   0  0     6
dmres              0  0     0  5   0  0     5
domcare            0  0     0  0   0  0     0
eth2001            0  0     0  0   0  0     0
eth_norecord       0  0     0  0   0  0     0
eth_notgiptref     0  0     0  0   0  0     0
eth_notstated      0  0     0  0   0  0     0
hhld_imdef         0  0     0  0   0  0     0
immdx_cov          0  0     0  0   0  0     0
immrx              0  0     0  0   0  0     0
learndis           0  0     0  0   0  0     0
longres            0  0     0  0   0  0     0
non_eth2001        0  0     0  0   0  0     0
nonshield          0  0     0  0   0  0     0
notcarer           0  0     0  0   0  0     0
nursehome          0  0     0  0   0  0     0
preg               0  0     0  0   0  0     0
pregdel            0  0     0  0   0  0     0
resp_cov           0  0     0  0   0  0     0
sev_mental         0  0     0  0   0  0     0
sev_obesity        0  0     0  0   0  0     0
shield             0  0     0  0   0  0     0
smhres             0  0     0  0   0  0     0
spln_cov           0  0     0  0   0  0     0